# LLM Batch Helper Tutorial

Welcome to the comprehensive tutorial for the LLM Batch Helper package! This notebook will walk you through all the features and capabilities of the package.

## What is LLM Batch Helper?

LLM Batch Helper is a Python package that enables efficient batch submission of prompts to LLM APIs with:
- **Simplified API** - No async/await syntax needed! 🎉
- **Async processing** handled implicitly for faster execution
- **Response caching** to avoid redundant API calls
- **Multiple input formats** (lists, files)
- **Custom verification** for response quality
- **Retry logic** with exponential backoff and detailed logging
- **Progress tracking** with progress bars
- **Jupyter notebook support** - works seamlessly without event loop issues

## 🎉 New in v0.3.0

- **Simplified Interface**: `process_prompts_batch()` now works without async/await
- **Jupyter Compatible**: No more event loop management needed
- **Detailed Retry Logging**: See exactly what happens during retries
- **Backward Compatibility**: Original async API still available

## Prerequisites

Before running this tutorial, make sure you have:
1. Installed the package: `pip install llm_batch_helper`
2. Set up your OpenAI API key: `export OPENAI_API_KEY="your-api-key"`

Let's get started! 🚀


In [ ]:
!pip install llm_batch_helper

In [ ]:
# Import required modules
import os
import json
from pathlib import Path

# Import the LLM Batch Helper components
from llm_batch_helper import LLMConfig, process_prompts_batch, LLMCache

# Check if OpenAI API key is set
if not os.environ.get("OPENAI_API_KEY"):
    print("⚠️  WARNING: OPENAI_API_KEY not found!")
    print("Please set your API key: export OPENAI_API_KEY='your-api-key'")
else:
    print("✅ OpenAI API key is set!")
    
print("🎉 Note: No more asyncio imports needed - the API is now simplified!")
    
print("🎉 LLM Batch Helper imported successfully!")


✅ OpenAI API key is set!
🎉 LLM Batch Helper imported successfully!


## 🆕 What's New in v0.3.0

This tutorial has been updated to showcase the **simplified API**! Here's what changed:

### ✅ Before (v0.2.0 and earlier)
```python
import asyncio
from llm_batch_helper import process_prompts_batch

async def main():
    results = await process_prompts_batch(...)
    return results

results = asyncio.run(main())
```

### 🎉 After (v0.3.0+)
```python
from llm_batch_helper import process_prompts_batch

# That's it - no async/await needed!
results = process_prompts_batch(...)
```

### 🔍 New Retry Logging
You'll also see detailed logging when retries happen:
```
🔄 [14:23:15] Retry attempt 1/5:
   Error: AuthenticationError (status: 401)
   Message: Incorrect API key provided...
   Waiting 4.0s before next attempt...
```

Let's dive into the tutorial! 🚀


# 1. Basic Usage

Let's start with a simple example: processing a list of prompts using the OpenAI API.

## Configuration

First, we need to create an `LLMConfig` object that defines how we want to interact with the API:


In [ ]:
# Create a basic configuration
config = LLMConfig(
    model_name="gpt-4o-mini",          # OpenAI model to use
    temperature=1.0,                   # Controls randomness (0.0 = deterministic, 1.0 = very random)
    max_completion_tokens=100,         # Maximum tokens in response (preferred over max_tokens)
    max_concurrent_requests=5,         # Number of parallel requests
    max_retries=3                      # Number of retries on failure
)

print("✅ Configuration created!")
print(f"Model: {config.model_name}")
print(f"Temperature: {config.temperature}")
print(f"Max tokens: {config.max_tokens}")
print(f"Max concurrent requests: {config.max_concurrent_requests}")


✅ Configuration created!
Model: gpt-4o-mini
Temperature: 0.7
Max tokens: 100
Max concurrent requests: 5


In [ ]:
# Define a list of prompts to process
prompts = [
    "What is the capital of France? Answer briefly.",
    "What is 2+2? Answer briefly.",
    "Who wrote 'Romeo and Juliet'? Answer briefly.",
    "What is the largest planet in our solar system? Answer briefly.",
    "What programming language is this tutorial written for? Answer briefly."
]

print(f"📝 Processing {len(prompts)} prompts...")

# 🎉 NEW: Process the prompts with simplified API - no async/await needed!
results = process_prompts_batch(
    config=config,
    provider="openai",
    prompts=prompts,
    cache_dir="tutorial_cache",
    desc="Basic Example"
)

print(f"\n✅ Processed {len(results)} prompts!")
print("📋 Results:")
for prompt_id, response in results.items():
    status = "[CACHE]" if response.get("from_cache") else "[GENERATED]"
    if "error" in response:
        print(f"{status} {prompt_id}: ERROR - {response['error']}")
    else:
        print(f"{status} {prompt_id}: {response['response_text']}")


📝 Processing 5 prompts...


Basic Example: 100%|██████████| 5/5 [00:01<00:00,  2.52it/s]


✅ Processed 5 prompts!
📋 Results:
[GENERATED] 87867a835fb23e91329cee573f1d5aef: The capital of France is Paris.
[GENERATED] fcf08bf32b9d323b77ffabe22d9b0ff3: 'Romeo and Juliet' was written by William Shakespeare.
[GENERATED] 6125526fef186f25b943810b334fe44a: The largest planet in our solar system is Jupiter.
[GENERATED] 99fe35a00746182c5f4aac010ebaff4b: 2 + 2 = 4.
[GENERATED] f81541ba5472630e22211367b36be7f6: I would need to see the tutorial or have information about it to determine the programming language it is written for. Please provide more details.


# 2. File-Based Processing

Instead of defining prompts in code, you can organize them in text files. This is especially useful for:
- Large sets of prompts
- Collaborative prompt development
- Version control of prompts
- Easier prompt management

Let's create some example prompt files and process them:


In [4]:
# Create a directory for prompt files
prompts_dir = Path("tutorial_prompts")
prompts_dir.mkdir(exist_ok=True)

# Create example prompt files
prompt_files = {
    "science_fact": "Tell me an interesting fact about black holes. Keep it under 50 words.",
    "creative_writing": "Write a short poem about coding. Make it fun and rhyming.",
    "problem_solving": "Explain how to solve a Rubik's cube in 3 simple steps.",
    "philosophy": "What is the meaning of life according to different philosophical traditions?",
    "technology": "Explain quantum computing to a 10-year-old."
}

# Write prompts to files
for filename, content in prompt_files.items():
    filepath = prompts_dir / f"{filename}.txt"
    with open(filepath, "w") as f:
        f.write(content)

print(f"📁 Created {len(prompt_files)} prompt files in '{prompts_dir}'")
print("📋 Files created:")
for filename in prompt_files.keys():
    print(f"  - {filename}.txt")
    
# List the actual files created
print(f"\n📂 Files in {prompts_dir}:")
for file in prompts_dir.glob("*.txt"):
    print(f"  - {file.name}")


📁 Created 5 prompt files in 'tutorial_prompts'
📋 Files created:
  - science_fact.txt
  - creative_writing.txt
  - problem_solving.txt
  - philosophy.txt
  - technology.txt

📂 Files in tutorial_prompts:
  - philosophy.txt
  - creative_writing.txt
  - problem_solving.txt
  - technology.txt
  - science_fact.txt


In [ ]:
# Process the prompt files with simplified API
file_results = process_prompts_batch(
    config=config,
    provider="openai",
    input_dir=str(prompts_dir),  # Directory containing .txt files
    cache_dir="tutorial_cache",
    desc="File-based Processing"
)

print(f"✅ Processed {len(file_results)} files!")
print("📋 Results:")
for prompt_id, response in file_results.items():
    status = "[CACHE]" if response.get("from_cache") else "[GENERATED]"
    if "error" in response:
        print(f"{status} {prompt_id}: ERROR - {response['error']}")
    else:
        print(f"{status} {prompt_id}: {response['response_text'][:100]}...")  # Show first 100 chars


File-based Processing: 100%|██████████| 5/5 [00:03<00:00,  1.60it/s]

✅ Processed 5 files!
📋 Results:
[GENERATED] science_fact: Black holes can warp time and space due to their immense gravity. Near a black hole, time slows down...
[GENERATED] technology: Sure! Imagine you have a really big box of LEGO bricks. Each brick can be either red or blue, and yo...
[GENERATED] creative_writing: In a world of zeros, ones, and code,  
A coder’s journey, a winding road.  
With fingers flying, key...
[GENERATED] problem_solving: Solving a Rubik's Cube can be complex, but here's a simplified approach broken down into three essen...
[GENERATED] philosophy: The meaning of life has been a central question in philosophy, and various traditions offer differen...


# 3. Custom Verification

Sometimes you want to ensure the quality of responses before accepting them. The LLM Batch Helper supports custom verification functions, for example, you can check:

- Check response length
- Validate content format
- Ensure specific keywords are present/absent
- Implement custom quality checks

If verification fails, the system will retry with a new request. Let's create a custom verification function:


In [ ]:
# Define a custom verification function
def quality_verification(prompt_id, llm_response_data, original_prompt_text, **kwargs):
    """
    Custom verification function that checks:
    1. Response is not empty
    2. Response meets minimum length requirement
    """
    
    response_text = llm_response_data.get("response_text", "")
    
    # Check 1: Not empty
    if not response_text.strip():
        print(f"❌ {prompt_id}: Empty response")
        return False
    
    # Check 2: Minimum length
    min_length = kwargs.get("min_length", 20)
    if len(response_text) < min_length:
        print(f"❌ {prompt_id}: Response too short ({len(response_text)} < {min_length})")
        return False
    
    print(f"✅ {prompt_id}: Verification passed")
    return True

# Create config with custom verification
verified_config = LLMConfig(
    model_name="gpt-4o-mini",
    temperature=1.0,
    max_completion_tokens=150,
    max_concurrent_requests=3,
    max_retries=3,
    verification_callback=quality_verification,
    verification_callback_args={"min_length": 30}
)

print("✅ Configuration with custom verification created!")


✅ Configuration with custom verification created!


In [ ]:
# Test prompts for verification
verification_test_prompts = [
    "What is the square root of 64? Explain how you calculated it.",
    "Write a detailed explanation of how photosynthesis works.",
    "Describe the process of machine learning in simple terms.",
]

# Process with verification using simplified API
print("🔍 Testing custom verification...")
verification_results = process_prompts_batch(
    config=verified_config,
    provider="openai",
    prompts=verification_test_prompts,
    cache_dir="tutorial_cache",
    desc="Verification Example"
)

print(f"\n✅ Verification test completed!")
print("📋 Results:")
for prompt_id, response in verification_results.items():
    status = "[CACHE]" if response.get("from_cache") else "[GENERATED]"
    if "error" in response:
        print(f"{status} {prompt_id}: ERROR - {response['error']}")
    else:
        print(f"{status} {prompt_id}: {response['response_text'][:80]}...")  # Show first 80 chars


🔍 Testing custom verification...


Verification Example:   0%|          | 0/3 [00:00<?, ?it/s]

✅ 215832376ebe50e1f3fc8d5417ab3cb9: Verification passed
✅ de61d1e78c4495cad5c9281947620db4: Verification passed


Verification Example: 100%|██████████| 3/3 [00:03<00:00,  1.29s/it]

✅ a50c47dd8f661a6b85fbd251004c986b: Verification passed

✅ Verification test completed!
📋 Results:
[CACHE] 215832376ebe50e1f3fc8d5417ab3cb9: The square root of 64 is 8. 

To calculate the square root, you can think of it ...
[CACHE] de61d1e78c4495cad5c9281947620db4: Photosynthesis is a biochemical process through which green plants, algae, and s...
[GENERATED] a50c47dd8f661a6b85fbd251004c986b: Sure! Machine learning is a way for computers to learn from data and make decisi...


# 4. Caching Features

One of the most powerful features of LLM Batch Helper is automatic response caching. This:

- **Saves money** by avoiding redundant API calls
- **Improves speed** by reusing previous responses
- **Enables experimentation** without re-running expensive operations
- **Supports reproducibility** in research and development

Let's explore the caching functionality:


In [ ]:
# Explore the cache directory
cache_dir = Path("tutorial_cache")
print(f"📁 Cache directory: {cache_dir}")
print(f"📂 Cache exists: {cache_dir.exists()}")

if cache_dir.exists():
    cache_files = list(cache_dir.glob("*.json"))
    print(f"📄 Number of cached responses: {len(cache_files)}")
    
    # Show some cache files
    print("\n📋 Cached responses:")
    for i, cache_file in enumerate(cache_files[:5]):  # Show first 5
        print(f"  {i+1}. {cache_file.name}")
    
    if len(cache_files) > 5:
        print(f"  ... and {len(cache_files) - 5} more")
    
    # Show content of one cache file
    if cache_files:
        sample_file = cache_files[0]
        print(f"\n📄 Sample cache file ({sample_file.name}):")
        with open(sample_file, 'r') as f:
            cache_content = json.load(f)
            print(f"  Prompt: {cache_content.get('prompt_input', 'N/A')[:60]}...")
            print(f"  Response: {cache_content.get('llm_response', {}).get('response_text', 'N/A')[:60]}...")
            print(f"  Cached at: {cache_content.get('llm_response', {}).get('timestamp', 'N/A')}")
else:
    print("🔍 No cache directory found yet - it will be created when you run examples!")


In [8]:
import hashlib
# Cache Management
cache = LLMCache(cache_dir="tutorial_cache")

# Demonstrate cache usage
print("🔧 Cache Management Examples:")

# Check if a specific prompt is cached
sample_prompt = "What is the capital of France? Answer briefly."
prompt_id = hashlib.sha256(sample_prompt.encode()).hexdigest()[:32]

# Try to get cached response
cached_response = cache.get_cached_response(prompt_id)
if cached_response:
    print(f"✅ Found cached response for prompt ID: {prompt_id}")
    print(f"   Response: {cached_response['llm_response']['response_text']}")
else:
    print(f"❌ No cached response found for prompt ID: {prompt_id}")

# Cache operations
print(f"\n🛠️  Cache Operations:")
print(f"   - Cache directory: {cache.cache_dir}")
print(f"   - Cache exists: {cache.cache_dir.exists()}")

# Option to clear cache (uncomment if needed)
# cache.clear_cache()
# print("🗑️  Cache cleared!")

# Show cache statistics
if cache.cache_dir.exists():
    cache_files = list(cache.cache_dir.glob("*.json"))
    total_size = sum(f.stat().st_size for f in cache_files)
    print(f"📊 Cache Statistics:")
    print(f"   - Files: {len(cache_files)}")
    print(f"   - Total size: {total_size / 1024:.2f} KB")
    print(f"   - Average file size: {total_size / len(cache_files) / 1024:.2f} KB" if cache_files else "   - Average file size: 0 KB")


🔧 Cache Management Examples:
✅ Found cached response for prompt ID: 87867a835fb23e91329cee573f1d5aef
   Response: The capital of France is Paris.

🛠️  Cache Operations:
   - Cache directory: tutorial_cache
   - Cache exists: True
📊 Cache Statistics:
   - Files: 13
   - Total size: 7.59 KB
   - Average file size: 0.58 KB
